In [ ]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
from google.colab import files
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False


In [ ]:
uploaded = files.upload() 
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn]))) 
data = pd.read_csv('/content/integrated_data.csv')
data['new_Date'] = pd.to_datetime(data['date'])
print(data.head())
print('\n')
print(data.info())
print('\n')
#첫번째 컬럼을 날짜형식 컬럼으로 변경 
print(type(data['new_Date'][0]))
data.drop('date', axis = 1, inplace=True)
data.set_index('new_Date', inplace=True)
print(data.head())
print('\n')
print(data.info())

In [ ]:
# 5열까지 출력 
data.head(5)


In [ ]:
#data['house_loan'].plot()
plt.title("household_loan")
plt.plot(data['house_loan'])
plt.show()
plt.title("consuming_price_idx")
plt.plot(data['consumer_price'])
plt.show()
plt.title("building_permitted_area")
plt.plot(data['building_space'])
plt.show()
plt.title("Interest_rate")
plt.plot(data['interest_idx'])
plt.show()
plt.title("housing price_idx")
plt.plot(data['housing_price'])
plt.show()


In [ ]:
data.head()
economic_stat = ['consumer_price','house_loan','building_space','interest_idx','housing_price']
features = data[economic_stat]
print(features.head(156))
print(features.values)
features.plot(subplots=True)
plt.show()

In [ ]:
#훈련데이터 분할 
#2008-01-01 ~ 2018-01-01 120개월을훈련데이터로 분할 
# 각 데이터 원시값-평균/표준편차 => 데이터셋 표준화 
TRAIN_SPLIT =120
tf.random.set_seed(13)
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)#각 열에 대한120인덱스 까지의  평균값 
data_std = dataset[:TRAIN_SPLIT].std(axis=0)# 각 열에 대한 표준편차차
print(data_std)
dataset = (dataset-data_mean)/data_std
print(dataset)

In [ ]:
#신경망 모델 훈련에 사용한 특정시간 윈도우 데이터 확보를 위한 메소드 
#history: 과거데이터 크기
#target size 모델이 미래를 예측할 크기 
# step값이 주어지면 step에 맞게 데이터를 샘플링 
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []          
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        print("dataset len %d",len(dataset))
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index+1):
        indices = range(i - history_size, i, step) 
        print(indices)
        data.append(dataset[indices])#
  
        if single_step:
          labels.append(target[i + target_size])
          print(label)
          print('타겟 데이터')
          print(target[i + target_size])
        else:
          labels.append(target[i:i + target_size])
                
    return np.array(data), np.array(labels)

In [ ]:
past_history = 24
future_target = 6
STEP = 1
"""
데이터 샘플링 
현재 보유중인 데이터: 월단위 
데이터 스플릿 120개월, 총 데이터 156 13년 20081.1 2020121
샘플링 단위 1개월 
과거 2년(24개월) 단위로 3개월 씩 샘플링 
타겟 1개월 예측 
range(0, 24, 1) 
range(1, 25, 1)
range(2, 26, 1)
range(3, 27, 3)
range(4, 28, 3)
range(5, 29, 3)
range(6, 30, 3)
range(7, 31, 3)
range(8, 32, 3)
학습 타겟 데이터: 훈련데이터가 매 샘플마다 저장될 때 레이블 데이터는 
해당 샘플의 마지막 인덱스+1(24일 때 25, 바로 다음달 ) 한 값을 타겟데이터로 저장 
"""

x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 4], 0, TRAIN_SPLIT, past_history, future_target, STEP)#0 ~ 30000
print('-----------------------------------')

# 예측해야할 데이터셋  11개 윈도우 120 ~ 154
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 4], TRAIN_SPLIT, None, past_history, future_target, STEP)#30000 ~ end

print('Single window of past history : {}'.format(x_train_multi[0].shape))
print(y_train_multi)
print('================================================================')
print('\n Target temperature to predict : {}'.format(y_val_multi[0].shape))
print(x_val_multi)
print('y')
print(y_val_multi)
# 처음부터 약2년까지의  3개월 단위 표준화 데이터셋 5개의 컬럼, 8개 행.

# 훈련데이터 차원 130,8,5..?

In [ ]:
BATCH_SIZE = 128##144 #128#256
#Batch는 일괄 처리되는 작업의 양이다. 위에서 설명된 data_size를 한번에 처리하는 갯수를 의미하며, 
#batch크기에 의해 weight 변화가 일어나며 batch크기 단위로 data loading 함수도 구현이 가능하다. 
#주의할점은 data_size/batch_size일때 나머지가 없어야 한다.
BUFFER_SIZE = 4096 #데이터 14000
EVALUATION_INTERVAL = 18
EPOCHS = 100
# 입력된 텐서로부터 slices를 생성한다. 
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
count =0
for element in train_data_multi:
  print("훈련데이터 슬라이스")
  count = count+1
  print(element)
print(count)  
  # 입력된 텐서로부터 slices를 생성한다. 무기한 반복
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
# 데이터 캐싱, 훈련데이터 셔플 
val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi)) #11,9,5 11,0
#.astype(float)
#.astype(float)
for element in val_data_multi:
  print("평가데이터 슬라이스")
  print(element)
print("이;거뭕ㄷ데")
print(val_data_multi)
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()



print(x_train_multi.shape) #96,25,5
print(x_train_multi.shape[-2:])

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
multi_step_model = tf.keras.models.Sequential()
#입력데이터 형식: 소비자물가지수, 가계대출금, 금리, 건축허가면적, 주택가격지수  입력형식 [8,5]
multi_step_model.add(tf.keras.layers.LSTM(32, return_sequences=True, input_shape=x_train_multi.shape[-2:])) #,dropout=0.5
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(6))

multi_step_model.compile(optimizer=tf.keras.optimizers.Adam(), loss = root_mean_squared_error)# 에러 절댓값 평균ingle_step_model.add(tf.keras.layers.Dense(1))
    #Adam()RMSprop()
#print(len(single_step_model.layers))#2
#훈련시작 배치크기 50회 학습, 
for x, y in val_data_multi.take(3):
  print(x.shape)
  print('이게뭔데 그래서')
  print(multi_step_model.predict(x).shape)
        
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_multi,
                                            validation_steps=3)
#steps_per_epoch [트레이닝데이터수/배치사이즈]
#validation_steps validation data수/배치사이즈


print(multi_step_history)
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

    plt.show()

plot_train_history(multi_step_history,
                   'Single Step Training and Validation Loss')

In [ ]:
def create_time_steps(length):
    return list(range(-length, 0))


def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12,6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)
  plt.plot(num_in, np.array(history[:,4]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',label='Predictied Future')
  plt.legend(loc='upper left')
  plt.show()

for x,y in train_data_multi.take(1):
  multi_step_plot(x[0],y[0], np.array([0]))      
for x,y in val_data_multi.take(3):
  multi_step_plot(x[0],y[0], multi_step_model.predict(x)[0])
  print(x.shape)
  print(x)

In [ ]:

plt.figure(figsize=(12,6))
plt.title("Pridicting All time Housing price")
plt.plot(data['housing_price'])
pridict = []
for x,y in train_data_multi.take(1):
  print(multi_step_model.predict(x).shape)
  print(multi_step_model.predict(x))
  #for window in range(96):
  #plt.show()
  print(x.shape)
print("---------원본데이터-------------------------------") 
print(data.shape)
print(data[:,4])

#  if prediction.any():

#    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',label='Predictied Future')
#plt.legend(loc='upper left')